In [ ]:
from pathlib import Path
import struct

edf_path = Path("data/demo/aaaaaaar_00000001.edf")

def parse_edf_header(file_path):
    with open(file_path, "rb") as f:
        header = f.read(256)
        # EDF fixed header parsing
        version = header[0:8].decode("ascii").strip()
        patient_id = header[8:88].decode("ascii").strip()
        recording_id = header[88:168].decode("ascii").strip()
        start_date = header[168:176].decode("ascii").strip()
        start_time = header[176:184].decode("ascii").strip()
        header_bytes = int(header[184:192].decode("ascii").strip())
        reserved = header[192:236].decode("ascii").strip()
        n_data_records = int(header[236:244].decode("ascii").strip())
        duration_data_record = float(header[244:252].decode("ascii").strip())
        n_signals = int(header[252:256].decode("ascii").strip())

        # Now read per-signal headers (n_signals * 256 bytes)
        signal_labels = []
        transducer_types = []
        physical_dimensions = []
        physical_mins = []
        physical_maxs = []
        digital_mins = []
        digital_maxs = []
        prefilterings = []
        n_samples_per_record = []
        reserveds = []

        def read_signal_field(offset, length):
            f.seek(256 + offset * n_signals)
            return [f.read(length).decode("ascii").strip() for _ in range(n_signals)]

        f.seek(256)
        labels = [f.read(16).decode("ascii").strip() for _ in range(n_signals)]
        transducer_types = [f.read(80).decode("ascii").strip() for _ in range(n_signals)]
        physical_dimensions = [f.read(8).decode("ascii").strip() for _ in range(n_signals)]
        physical_mins = [float(f.read(8).decode("ascii").strip()) for _ in range(n_signals)]
        physical_maxs = [float(f.read(8).decode("ascii").strip()) for _ in range(n_signals)]
        digital_mins = [int(f.read(8).decode("ascii").strip()) for _ in range(n_signals)]
        digital_maxs = [int(f.read(8).decode("ascii").strip()) for _ in range(n_signals)]
        prefilterings = [f.read(80).decode("ascii").strip() for _ in range(n_signals)]
        n_samples_per_record = [int(f.read(8).decode("ascii").strip()) for _ in range(n_signals)]
        reserveds = [f.read(32).decode("ascii").strip() for _ in range(n_signals)]

    return {
        "version": version,
        "patient_id": patient_id,
        "recording_id": recording_id,
        "start_date": start_date,
        "start_time": start_time,
        "header_bytes": header_bytes,
        "n_data_records": n_data_records,
        "duration_data_record": duration_data_record,
        "n_signals": n_signals,
        "labels": labels,
        "transducer_types": transducer_types,
        "physical_dimensions": physical_dimensions,
        "physical_mins": physical_mins,
        "physical_maxs": physical_maxs,
        "digital_mins": digital_mins,
        "digital_maxs": digital_maxs,
        "prefilterings": prefilterings,
        "n_samples_per_record": n_samples_per_record,
        "reserveds": reserveds,
    }

edf_info = parse_edf_header(edf_path)
edf_info["n_signals"], edf_info["labels"][:10]